In [6]:
import math
import numpy as np
import seaborn as sns
from time import time
import pandas as pd
import pymcmcstat
import matplotlib.pyplot as plt

In [ ]:
def model_magnitud(theta, data):
    mag_v_sun = -26.72

    phase_angle = data['obs_phase_angle']
    range_sat = data['range_sat']
    cross_section = theta['cross_section']
    albedo = theta['albedo'] 
    mix_coef = theta['mix_coeff']

   # Change degrees to radian
    degtorad = lambda x : x*np.pi/180.

   # Definition coefficients of equation
    coe_1 = 2/(3*np.pi)
    coe_2 = 1/(4*np.pi)

    phase_angle_rad = degtorad(phase_angle)	

    part1 = (coe_1 *((np.pi - phase_angle_rad)*np.cos(phase_angle_rad) - np.sin(phase_angle_rad))) - coe_2
	
    mag = mag_v_sun - 2.5 * np.log10(cross_section * albedo * (mix_coef * part1) + coe_2) + 5*np.log10(range_sat)
    return mag

In [ ]:
def sum_squares(theta, data):

    rest = data['mag_observation'] - model_magnitud(theta, data)
    print(rest)
    return (rest ** 2).sum(axis=0)

def residual_calc(theta, data_x, data_y):
    residual = data_y - model_magnitud(theta, data_x)
    return residual

### Read Data and create new dataframe

In [13]:
path = '/home/kero/Documents/PhD/Re-analyse/result_analyse.csv'
data_sat = pd.read_csv(path)

data_in = pd.DataFrame()
data_in['obs_phase_angle'] = data_sat['obs_phase_angle']
data_in['range_sat'] = data_sat['range_sat']
data_in['mag_observation'] = data_sat['mag_observation']
data_in.shape

(144, 3)

In [14]:
theta0 = dict(
        cross_section=0.5,
        albedo=0.,
        mix_coef=0.5
        )

mean = 0.8
std_dev = 0.35
num_reps = 144
num_simulations = 1000

cross_section = np.random.normal(mean, std_dev, num_reps).round(2)

In [15]:
mean = 0.55
std_dev = 0.2
num_reps = 144
num_simulations = 1000

albedo = np.random.normal(mean, std_dev, num_reps).round(2)

In [16]:
mean = 0.75
std_dev = 0.12
num_reps = 144
num_simulations = 1000

mix_coef = np.random.normal(mean, std_dev, num_reps).round(2)


In [17]:
np.shape(mix_coef)

(144,)

In [ ]:
for i in range(num_simulations):
    # Choose random inputs for the sales targets and percent to target
    mix_coef = np.random.normal(mean, std_dev, num_reps).round(2)
    albedo = np.random.normal(mean, std_dev, num_reps).round(2)
    cross_section = np.random.normal(mean, std_dev, num_reps).round(2)


    df = pd.DataFrame(index=range(num_reps), data={'mix_coef': mix_coef,
                                                'albedo': albedo,
                                                'cross_section' : cross_section})

    model_value = model_magnitud(df, data_in)

    residual_value = model_value - data_in['mag_observation']

    result = pd.DataFrame(index=range(num_reps), data={'Pct_To_Target': pct_to_target,
                                                   'Sales_Target': sales_target})

